In [34]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
import datetime

In [35]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [36]:
# Define database and collection
db = client.project_02
collection = db.gpu

In [37]:
# URL of page to be scraped
url = 'https://www.newegg.com/p/pl?N=100007709%20601357248'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [40]:
# Examine the results, then determine element that contains sought info on Newegg
# results are returned as an iterable list
results = soup.find_all('div', class_='item-cell')
# Loop through returned results
for result in results:
    try:
        # Identify and return title of listing
        title = result.find('a', class_='item-title').text
        # Identify and return link to listing
        link = result.a['href']
        # Identify and return price of listing
        price_string = result.find('li', class_='price-current')
        dollars = price_string.find('strong').text.replace(',','')
        cents = price_string.find('sup').text
        price = float(dollars + cents)

        # Run only if title, price, and link are available
        if (title and link and price):
            # Print results
            print('-------------')
            print(title)
            print(price)
            print(link)
            print(datetime.datetime.now())

            # Dictionary to be inserted as a MongoDB document
            post = {
                'title': title,
                'price': price,
                'url': link,
                'time': datetime.datetime.now()
            }
            collection.insert_one(post)

    except Exception as e:
        print(e)

-------------
MSI Gaming GeForce RTX 3090 24GB GDDR6X PCI Express 4.0 SLI Support Video Card RTX 3090 GAMING X TRIO 24G
3379.69
https://www.newegg.com/msi-geforce-rtx-3090-rtx-3090-gaming-x-trio-24g/p/N82E16814137595
2021-12-07 21:14:31.051173
-------------
ASUS ROG Strix GeForce RTX 3090 24GB GDDR6X PCI Express 4.0 SLI Support Video Card ROG-STRIX-RTX3090-O24G-WHITE
3689.99
https://www.newegg.com/asus-geforce-rtx-3090-rog-strix-rtx3090-o24g-white/p/N82E16814126482
2021-12-07 21:14:31.053175
-------------
MSI Ventus GeForce RTX 3090 24GB GDDR6X PCI Express 4.0 SLI Support Video Card RTX 3090 VENTUS 3X 24G OC
3159.99
https://www.newegg.com/msi-geforce-rtx-3090-rtx-3090-ventus-3x-24g-oc/p/N82E16814137596
2021-12-07 21:14:31.053175
-------------
GIGABYTE AORUS GeForce RTX 3090 XTREME WATERFORCE WB 24G Graphics Card, WATERFORCE Water Block Cooling System, 24GB 384-bit GDDR6X, GV-N3090AORUSX WB-24GD Video Card
3404.8
https://www.newegg.com/gigabyte-geforce-rtx-3090-gv-n3090aorusx-wb-24gd/p/

In [41]:
# URL of page to be scraped
url = 'https://www.bestbuy.com/site/searchpage.jsp?_dyncharset=UTF-8&browsedCategory=abcat0507002&id=pcat17071&iht=n&ks=960&list=y&qp=gpusv_facet%3DGraphics%20Processing%20Unit%20(GPU)~NVIDIA%20GeForce%20RTX%203090&sc=Global&st=categoryid%24abcat0507002&type=page&usc=All%20Categories'

agent = {'User-Agent':'Mozilla/5.0'}

# Retrieve page with the requests module
response = requests.get(url, headers=agent)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [44]:
# Examine the results, then determine element that contains sought info on BestBuy
# results are returned as an iterable list
results = soup.find_all('div', class_='list-item lv')
# Loop through returned results
for result in results:
    try:
        # Identify and return title of listing
        title = result.find('div', class_='sku-title').find('a').text
#         # Identify and return link to listing
        link_string = result.a['href']
        link = 'https://www.bestbuy.com/' + link_string
#         # Identify and return price of listing
        price_string = result.find('span', class_='sr-only').text.split('$')[1]
        price = float(price_string.replace(',',''))

        # Run only if title, price, and link are available
        if (title and link and price):
            # Print results
            print('-------------')
            print(title)
            print(price)
            print(link)
            print(datetime.datetime.now())

            # Dictionary to be inserted as a MongoDB document
            post = {
                'title': title,
                'price': price,
                'url': link,
                'time': datetime.datetime.now()
            }
            collection.insert_one(post)

    except Exception as e:
        print(e)

-------------
NVIDIA GeForce RTX 3090 24GB GDDR6X  PCI Express 4.0 Graphics Card - Titanium and Black
1499.99
https://www.bestbuy.com//site/nvidia-geforce-rtx-3090-24gb-gddr6x-pci-express-4-0-graphics-card-titanium-and-black/6429434.p?skuId=6429434
2021-12-07 21:17:33.757505
-------------
ASUS - GeForce RTX 3090 24GB GDDR6X PCI Express 4.0 Strix Graphics Card - Black
2299.99
https://www.bestbuy.com//site/asus-geforce-rtx-3090-24gb-gddr6x-pci-express-4-0-strix-graphics-card-black/6432447.p?skuId=6432447
2021-12-07 21:17:33.757505
-------------
GIGABYTE - NVIDIA GeForce RTX 3090 GAMING OC 24GB GDDR6X PCI Express 4.0 Graphics Card
2199.99
https://www.bestbuy.com//site/gigabyte-nvidia-geforce-rtx-3090-gaming-oc-24gb-gddr6x-pci-express-4-0-graphics-card/6430623.p?skuId=6430623
2021-12-07 21:17:33.757505
-------------
ASUS - TUF RTX 3090 24GB GDDR6X PCI Express 4.0 Graphics Card - Black
2199.99
https://www.bestbuy.com//site/asus-tuf-rtx-3090-24gb-gddr6x-pci-express-4-0-graphics-card-black/64

In [43]:
# Display items in MongoDB collection
listings = db.gpu.find()

for listing in listings:
    print(listing)

{'_id': ObjectId('61b01507ae3cf05cf1057b47'), 'title': 'MSI Gaming GeForce RTX 3090 24GB GDDR6X PCI Express 4.0 SLI Support Video Card RTX 3090 GAMING X TRIO 24G', 'price': 3379.69, 'url': 'https://www.newegg.com/msi-geforce-rtx-3090-rtx-3090-gaming-x-trio-24g/p/N82E16814137595', 'time': datetime.datetime(2021, 12, 7, 21, 14, 31, 51000)}
{'_id': ObjectId('61b01507ae3cf05cf1057b48'), 'title': 'ASUS ROG Strix GeForce RTX 3090 24GB GDDR6X PCI Express 4.0 SLI Support Video Card ROG-STRIX-RTX3090-O24G-WHITE', 'price': 3689.99, 'url': 'https://www.newegg.com/asus-geforce-rtx-3090-rog-strix-rtx3090-o24g-white/p/N82E16814126482', 'time': datetime.datetime(2021, 12, 7, 21, 14, 31, 53000)}
{'_id': ObjectId('61b01507ae3cf05cf1057b49'), 'title': 'MSI Ventus GeForce RTX 3090 24GB GDDR6X PCI Express 4.0 SLI Support Video Card RTX 3090 VENTUS 3X 24G OC', 'price': 3159.99, 'url': 'https://www.newegg.com/msi-geforce-rtx-3090-rtx-3090-ventus-3x-24g-oc/p/N82E16814137596', 'time': datetime.datetime(2021, 